In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covidct/COVID-CT-MetaInfo.xlsx
/kaggle/input/covidct/CT_NonCOVID/65%4.jpg
/kaggle/input/covidct/CT_NonCOVID/54%1.jpg
/kaggle/input/covidct/CT_NonCOVID/51%1.jpg
/kaggle/input/covidct/CT_NonCOVID/1814.png
/kaggle/input/covidct/CT_NonCOVID/1924.png
/kaggle/input/covidct/CT_NonCOVID/709.png
/kaggle/input/covidct/CT_NonCOVID/889.png
/kaggle/input/covidct/CT_NonCOVID/45.jpg
/kaggle/input/covidct/CT_NonCOVID/20.jpg
/kaggle/input/covidct/CT_NonCOVID/6%0.jpg
/kaggle/input/covidct/CT_NonCOVID/69%1.jpg
/kaggle/input/covidct/CT_NonCOVID/227.png
/kaggle/input/covidct/CT_NonCOVID/39%0.jpg
/kaggle/input/covidct/CT_NonCOVID/36%1.jpg
/kaggle/input/covidct/CT_NonCOVID/579.png
/kaggle/input/covidct/CT_NonCOVID/1048.png
/kaggle/input/covidct/CT_NonCOVID/39%4.jpg
/kaggle/input/covidct/CT_NonCOVID/33%1.jpg
/kaggle/input/covidct/CT_NonCOVID/177.png
/kaggle/input/covidct/CT_NonCOVID/754.png
/kaggle/input/covidct/CT_NonCOVID/104.png
/kaggle/input/covidct/CT_NonCOVID/1312.png
/kaggle/input/covidct

In [1]:
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from glob import glob
import random
import torch.optim as optim
from sklearn.model_selection import KFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
torch.cuda.empty_cache()

In [5]:
ds_path = '/kaggle/input/covidct'

# **Data Transformation**

In [6]:
import PIL.ImageOps

def to_negative(img):
    img = PIL.ImageOps.invert(img)
    return img

class Negative(object):
    """Convert image to negative.

    Args:
        

    Returns:
        PIL Image: Negative version of the input.
         
    """
    def __init__(self):
        pass
    
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be converted to Negative.

        Returns:
            PIL Image: Negatived image.
        """
        return to_negative(img)

In [7]:
dataset = dset.ImageFolder(
    root=ds_path,
    transform=transforms.Compose([
               transforms.Resize((200,200)),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

In [8]:
from torchvision import datasets, transforms

train_transformer = transforms.Compose([
        transforms.RandomResizedCrop(size=(480, 480), scale=(0.5, 1.0)),  
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor()
    ])

# **Train-test split**

In [9]:
random.seed(123)
test_ratio=0.1

total_length = len(dataset)
test_length = int(test_ratio * total_length)
train_length = total_length - test_length

train_dataset, test_dataset = random_split(dataset, [train_length, test_length])

In [10]:
train_dataset.dataset = datasets.ImageFolder(
    root=train_dataset.dataset.root,
    transform=train_transformer,
    target_transform=train_dataset.dataset.target_transform
)

In [11]:
train_dataset.dataset

Dataset ImageFolder
    Number of datapoints: 746
    Root location: /kaggle/input/covidct
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(480, 480), scale=(0.5, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
               ToTensor()
           )

In [12]:
test_dataset.dataset

Dataset ImageFolder
    Number of datapoints: 746
    Root location: /kaggle/input/covidct
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [13]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [14]:
img, label = next(iter(train_loader))

print(img.shape, label.shape)

torch.Size([16, 3, 480, 480]) torch.Size([16])


# **Model 1**

In [15]:
import torch
import torch.nn as nn

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(256, 512, kernel_size=5, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(1024, 2048, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1)  # Assuming 2 classes for classification
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of the Model1
model_a = Model1()
# Print the model architecture
print(model_a)


Model1(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(1024, 2048, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classifier): Sequen

In [15]:
class Model1_b(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=5, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(256, 512, kernel_size=5, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=0),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(1024, 2048, kernel_size=5, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),  # Adding a dropout layer
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  # Adding a dropout layer
            nn.Linear(512, 1)  # Assuming 2 classes for classification
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of the Model1
model_b = Model1_b()
# Print the model architecture
print(model)


Model1(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
    (13): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): MaxPool2d(kerne

# **Model 2**

In [15]:
import torch
import torch.nn as nn

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()  # Adding a sigmoid activation
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of Model2
model2_a = Model2()
# Print the model architecture
print(model2_a)


Model2(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [37]:
class Model2_b(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  
            nn.Sigmoid()  # Adding a sigmoid activation
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of Model2
model2_b = Model2_b()
# Print the model architecture
print(model2)


Model2(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_fe

# **Model 3**

In [ ]:
import torch
import torch.nn as nn

class Model3_a(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of Model3
model3 = Model3_a()
# Print the model architecture
print(model3)


In [85]:
class Model3_b(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),  # Adding dropout for regularization
            nn.Linear(32, 1)  
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Create an instance of Model3
model3 = Model3_b()
# Print the model architecture
print(model3)


Model3(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [16]:
# model_a.to(device)
model2_a.to(device)
# model3_a.to(device)

Model2(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

# **VGG19**

In [15]:
import torchvision.models as models

In [16]:
vgg19 = models.vgg19(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 264MB/s]  


In [17]:
for param in vgg19.parameters():
    param.requires_grad = False

In [18]:
vgg19.classifier[6].in_features

4096

In [19]:
vgg19.classifier[6] = nn.Sequential(
    nn.Linear(vgg19.classifier[6].in_features, 512),
#     nn.BatchNorm1d(512),  # Adding batch normalization
    nn.ReLU(),
#     nn.Dropout(0.5),  # Adding dropout for regularization
    nn.Linear(512, 1)
)

In [20]:
vgg19=vgg19.to(device)

In [16]:
pip install torchsummary


Note: you may need to restart the kernel to use updated packages.


In [17]:
from torchsummary import summary

In [23]:
summary(vgg19,(3,200,200),32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 200, 200]           1,792
              ReLU-2         [32, 64, 200, 200]               0
            Conv2d-3         [32, 64, 200, 200]          36,928
              ReLU-4         [32, 64, 200, 200]               0
         MaxPool2d-5         [32, 64, 100, 100]               0
            Conv2d-6        [32, 128, 100, 100]          73,856
              ReLU-7        [32, 128, 100, 100]               0
            Conv2d-8        [32, 128, 100, 100]         147,584
              ReLU-9        [32, 128, 100, 100]               0
        MaxPool2d-10          [32, 128, 50, 50]               0
           Conv2d-11          [32, 256, 50, 50]         295,168
             ReLU-12          [32, 256, 50, 50]               0
           Conv2d-13          [32, 256, 50, 50]         590,080
             ReLU-14          [32, 256,

# **Resnet50**

In [18]:
resnet50 = models.resnet50(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 237MB/s]


In [19]:
for param in resnet50.parameters():
    param.requires_grad = False


In [20]:
resnet50.fc = nn.Sequential(
    nn.Linear(resnet50.fc.in_features, 512),
    nn.BatchNorm1d(512),  # Adding batch normalization
    nn.ReLU(),
    nn.Dropout(0.5),  # Adding dropout for regularization
    nn.Linear(512, 1)
)

In [21]:
resnet50=resnet50.to(device)

In [22]:
summary(resnet50, (3, 200, 200), 32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 100, 100]           9,408
       BatchNorm2d-2         [32, 64, 100, 100]             128
              ReLU-3         [32, 64, 100, 100]               0
         MaxPool2d-4           [32, 64, 50, 50]               0
            Conv2d-5           [32, 64, 50, 50]           4,096
       BatchNorm2d-6           [32, 64, 50, 50]             128
              ReLU-7           [32, 64, 50, 50]               0
            Conv2d-8           [32, 64, 50, 50]          36,864
       BatchNorm2d-9           [32, 64, 50, 50]             128
             ReLU-10           [32, 64, 50, 50]               0
           Conv2d-11          [32, 256, 50, 50]          16,384
      BatchNorm2d-12          [32, 256, 50, 50]             512
           Conv2d-13          [32, 256, 50, 50]          16,384
      BatchNorm2d-14          [32, 256,

In [23]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [24]:
def train(model, train_loader, device, loss_fn, optimizer):
    train_loss = 0.0
   
    
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        y_logits = model(X).squeeze()
        y_pred = torch.round(torch.sigmoid(y_logits))
        loss = loss_fn(y_pred, y.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    return train_loss / len(train_loader)

In [25]:
def validate(model, val_loader, device, loss_fn):
    val_loss = 0.0
    val_acc = 0.0
    nb_classes = 2
    confusion_matrix_val = torch.zeros(nb_classes, nb_classes)
    
    model.eval()
    
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            y_logits = model(X).squeeze()
            y_pred = torch.round(torch.sigmoid(y_logits))
            loss = loss_fn(y_pred, y.float())
            val_loss += loss.item()
            
            for t, p in zip(y.view(-1), y_pred.view(-1)):
                confusion_matrix_val[t.long(), p.long()] += 1
            
            val_acc += (y_pred == y).sum().item() / len(y)
    
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    
    return val_loss, val_acc, confusion_matrix_val

In [26]:
def calculate_metrics(confusion_matrix):
    true_positives = confusion_matrix[1, 1]
    false_positives = confusion_matrix[0, 1]
    false_negatives = confusion_matrix[1, 0]
    true_negatives = confusion_matrix[0,0]

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy= (true_positives+true_negatives)/(true_positives+false_positives+false_negatives+true_negatives)

    return precision, recall, f1_score, accuracy

In [37]:
def train_and_validate(model, train_dataset, device, loss_fn, optimizer, num_epochs):
    history = {
    'train_loss': [],
    'val_loss': [],
    'val_acc': [],
    'precision': [],  
    'recall': [],  
    'f1_score': []  
}
    
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for fold, (train_index, val_index) in enumerate(kf.split(train_dataset.dataset.imgs, train_dataset.dataset.targets)):
        
        train_dataset = torch.utils.data.Subset(dataset, train_index)
        val_dataset = torch.utils.data.Subset(dataset, val_index)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        for epoch in range(num_epochs):
            train_loss = train(model, train_loader, device, loss_fn, optimizer)
            val_loss, val_acc, confusion_matrix_val = validate(model, val_loader, device, loss_fn)

            precision, recall, f1_score,accuracy = calculate_metrics(confusion_matrix_val)

            accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1_score)
            
            
            print(f'Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs}')

            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            history['precision'].append(precision)
            history['recall'].append(recall)
            history['f1_score'].append(f1_score)
            
            
    
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    print(f'Average Accuracy: {avg_accuracy}')
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    
    return history, model



In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def test(model, test_loader, device):
    test_pred = []
    test_labels = []
    model.eval()
    with torch.inference_mode():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            preds = torch.round(torch.sigmoid(outputs))
            test_pred.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_labels, test_pred)
    precision = precision_score(test_labels, test_pred)
    recall = recall_score(test_labels, test_pred)
    f1 = f1_score(test_labels, test_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1


In [39]:
loss_fn = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.0001)


In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
print('Training the model')
history, trained_model = train_and_validate(resnet50, train_dataset, device, loss_fn, optimizer, 6)

Training the model
Fold 1, Epoch 1/6
Fold 1, Epoch 2/6
Fold 1, Epoch 3/6
Fold 1, Epoch 4/6
Fold 1, Epoch 5/6
Fold 1, Epoch 6/6
Fold 2, Epoch 1/6
Fold 2, Epoch 2/6
Fold 2, Epoch 3/6
Fold 2, Epoch 4/6
Fold 2, Epoch 5/6
Fold 2, Epoch 6/6
Fold 3, Epoch 1/6
Fold 3, Epoch 2/6
Fold 3, Epoch 3/6
Fold 3, Epoch 4/6
Fold 3, Epoch 5/6
Fold 3, Epoch 6/6
Fold 4, Epoch 1/6
Fold 4, Epoch 2/6
Fold 4, Epoch 3/6
Fold 4, Epoch 4/6
Fold 4, Epoch 5/6
Fold 4, Epoch 6/6
Fold 5, Epoch 1/6
Fold 5, Epoch 2/6
Fold 5, Epoch 3/6
Fold 5, Epoch 4/6
Fold 5, Epoch 5/6
Fold 5, Epoch 6/6
Average Accuracy: 0.5301640033721924
Average Precision: 0.5386028289794922
Average Recall: 0.8247923254966736
Average F1 Score: 0.6507608890533447


In [42]:
print('testing the model')
test(trained_model, test_loader, device)

testing the model
Accuracy: 0.5000
Precision: 0.5167
Recall: 0.7949
F1 Score: 0.6263


(0.5, 0.5166666666666667, 0.7948717948717948, 0.6262626262626263)

In [27]:
# print('Training the model')
# history2, trained_model2 = train_and_validate(model2, train_dataset, device, loss_fn, optimizer, 10)

In [49]:
print('testing the model')
test(trained_model2, test_loader, device)

testing the model
Accuracy: 0.5405
Precision: 0.5405
Recall: 1.0000
F1 Score: 0.7018


(0.5405405405405406, 0.5405405405405406, 1.0, 0.7017543859649124)

In [95]:
print('Training the model')
history3, trained_model3 = train_and_validate(model3, train_dataset, device, loss_fn, optimizer, 6)

Training the model
Fold 1, Epoch 1/10
Fold 1, Epoch 2/10
Fold 1, Epoch 3/10
Fold 1, Epoch 4/10
Fold 1, Epoch 5/10
Fold 1, Epoch 6/10
Fold 1, Epoch 7/10
Fold 1, Epoch 8/10
Fold 1, Epoch 9/10
Fold 1, Epoch 10/10
Fold 2, Epoch 1/10
Fold 2, Epoch 2/10
Fold 2, Epoch 3/10
Fold 2, Epoch 4/10
Fold 2, Epoch 5/10
Fold 2, Epoch 6/10
Fold 2, Epoch 7/10
Fold 2, Epoch 8/10
Fold 2, Epoch 9/10
Fold 2, Epoch 10/10
Fold 3, Epoch 1/10
Fold 3, Epoch 2/10
Fold 3, Epoch 3/10
Fold 3, Epoch 4/10
Fold 3, Epoch 5/10
Fold 3, Epoch 6/10
Fold 3, Epoch 7/10
Fold 3, Epoch 8/10
Fold 3, Epoch 9/10
Fold 3, Epoch 10/10
Fold 4, Epoch 1/10
Fold 4, Epoch 2/10
Fold 4, Epoch 3/10
Fold 4, Epoch 4/10
Fold 4, Epoch 5/10
Fold 4, Epoch 6/10
Fold 4, Epoch 7/10
Fold 4, Epoch 8/10
Fold 4, Epoch 9/10
Fold 4, Epoch 10/10
Fold 5, Epoch 1/10
Fold 5, Epoch 2/10
Fold 5, Epoch 3/10
Fold 5, Epoch 4/10
Fold 5, Epoch 5/10
Fold 5, Epoch 6/10
Fold 5, Epoch 7/10
Fold 5, Epoch 8/10
Fold 5, Epoch 9/10
Fold 5, Epoch 10/10
Average Accuracy: 0.467829

In [64]:
print('testing the model')
test(trained_model3, test_loader, device)

testing the model
Accuracy: 0.4595
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4594594594594595, 0.0, 0.0, 0.0)